In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Anxiety_disorder"
cohort = "GSE60190"

# Input paths
in_trait_dir = "../../input/GEO/Anxiety_disorder"
in_cohort_dir = "../../input/GEO/Anxiety_disorder/GSE60190"

# Output paths
out_data_file = "../../output/preprocess/Anxiety_disorder/GSE60190.csv"
out_gene_data_file = "../../output/preprocess/Anxiety_disorder/gene_data/GSE60190.csv"
out_clinical_data_file = "../../output/preprocess/Anxiety_disorder/clinical_data/GSE60190.csv"
json_path = "../../output/preprocess/Anxiety_disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
import json
import numpy as np
from typing import Dict, Any, Callable, Optional, List, Tuple

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from DLPFC
# using Illumina HumanHT-12 v3 microarray, which is suitable for our analysis
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Examining the sample characteristics dictionary to identify relevant rows

# For trait, we can use row 3 which has 'dx' (diagnosis) with values including 'Control', 'ED', and 'OCD'
trait_row = 3

# For age, we can use row 5 which has 'age' values
age_row = 5

# For gender, we can use row 7 which has 'Sex' values
gender_row = 7

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> int:
    """
    Convert anxiety disorder trait information to binary format.
    For Anxiety_disorder as the trait of interest, we consider OCD as 1 (case) and Control as 0 (control).
    Exclude other conditions like ED, MDD, etc.
    
    Args:
        value: The raw trait value from the dataset
    
    Returns:
        int: 1 for anxiety disorder (OCD), 0 for control, None for other conditions or missing values
    """
    if not value or ':' not in value:
        return None
    
    diagnosis = value.split(':', 1)[1].strip()
    
    # For anxiety disorder, we consider OCD patients as cases
    if diagnosis == 'OCD' or diagnosis == 'Tics':  # Tics can be related to anxiety disorders
        return 1
    elif diagnosis == 'Control':
        return 0
    else:
        return None  # Exclude other diagnoses like ED, Bipolar, MDD

def convert_age(value: str) -> float:
    """
    Convert age information to continuous format.
    
    Args:
        value: The raw age value from the dataset
    
    Returns:
        float: Age in years, or None if missing
    """
    if not value or ':' not in value:
        return None
    
    try:
        age_str = value.split(':', 1)[1].strip()
        return float(age_str)
    except (ValueError, TypeError):
        return None

def convert_gender(value: str) -> int:
    """
    Convert gender information to binary format (0 for female, 1 for male).
    
    Args:
        value: The raw gender value from the dataset
    
    Returns:
        int: 0 for female, 1 for male, or None if missing
    """
    if not value or ':' not in value:
        return None
    
    gender = value.split(':', 1)[1].strip()
    
    if gender == 'F':
        return 0
    elif gender == 'M':
        return 1
    else:
        return None

# 3. Save Metadata - Perform initial filtering
# Trait data is available since trait_row is not None
is_trait_available = trait_row is not None

# Use the validate_and_save_cohort_info function to save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step as clinical_data from previous step is required but not available
# This will be executed in a subsequent step when clinical_data is available


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll provide a properly formatted solution for this step:

```python
import os
import json
import pandas as pd
import numpy as np
from typing import Callable, Optional, Dict, Any

# Check what files are available in the directory
print(f"Checking files in: {in_cohort_dir}")
available_files = os.listdir(in_cohort_dir) if os.path.exists(in_cohort_dir) else []
print(f"Available files: {available_files}")

# For GEO datasets, we typically have matrix files and series_matrix files
matrix_files = [f for f in available_files if 'matrix' in f.lower()]
print(f"Matrix files found: {matrix_files}")

# Check if the gene expression data is available
is_gene_available = False
for file in available_files:
    if file.endswith('.soft') or file.endswith('.txt') or 'matrix' in file.lower():
        try:
            with open(os.path.join(in_cohort_dir, file), 'r') as f:
                content = f.read(10000)  # Read first 10000 characters
                # Look for indicators of gene expression data
                if any(term in content.lower() for term in ["gene_expression", "platform_id", "platform ="]):
                    is_gene_available = True
                    break
                # Filter out pure miRNA or methylation datasets
                if all(term in content.lower() for term in ["mirna", "microrna"]) and "gene expression" not in content.lower():
                    is_gene_available = False
                if "methylation" in content.lower() and "gene expression" not in content.lower():
                    is_gene_available = False
        except Exception as e:
            print(f"Error checking file {file}: {e}")

# Load sample characteristics if available
sample_characteristics = {}
clinical_data = None

# Try different file patterns for clinical data
possible_clinical_files = [
    os.path.join(in_cohort_dir, "clinical_data.csv"),
    os.path.join(in_cohort_dir, "GSE60190_series_matrix.txt"),
    os.path.join(in_cohort_dir, "series_matrix.txt")
]

for file_path in possible_clinical_files:
    if os.path.exists(file_path):
        print(f"Found clinical data file: {file_path}")
        if file_path.endswith('.csv'):
            clinical_data = pd.read_csv(file_path)
        else:
            # For series_matrix files, we need to parse the !Sample_characteristics lines
            try:
                with open(file_path, 'r') as f:
                    lines = f.readlines()
                
                # Extract sample characteristics lines
                char_lines = [line.strip() for line in lines if line.startswith("!Sample_characteristics")]
                
                # Parse sample characteristics
                for i, line in enumerate(char_lines):
                    # Extract values after the equals sign
                    values = [part.split("=")[1].strip() if "=" in part else part.strip() 
                             for part in line.split("\t")[1:]]
                    if values:
                        sample_characteristics[i] = values
                
                # Also create a dataframe from the characteristics
                if sample_characteristics:
                    # Convert to a format suitable for a dataframe
                    samples = list(set([val for sublist in sample_characteristics.values() for val in sublist]))
                    clinical_data = pd.DataFrame(index=range(len(sample_characteristics)), 
                                               columns=['characteristic'] + samples)
                    for i, values in sample_characteristics.items():
                        clinical_data.iloc[i, 0] = f"characteristic_{i}"
                        for val in values:
                            clinical_data.loc[i, val] = val
            except Exception as e:
                print(f"Error parsing series matrix file: {e}")
        break

if clinical_data is None and sample_characteristics:
    # If we have sample characteristics but no dataframe, create one
    clinical_data = pd.DataFrame()
    for i, values in sample_characteristics.items():
        clinical_data.loc[i, 'characteristic'] = f"characteristic_{i}"
        for val in values:
            clinical_data.loc[i, val] = val

# Also check for a background info file
background_info = ""
background_path = os.path.join(in_cohort_dir, "background_info.txt")
if os.path.exists(background_path):
    with open(background_path, 'r') as f:
        background_info = f.read()
    print("\nBackground Info:")
    print(background_info)

# Print sample characteristics for analysis
print("\nSample Characteristics:")
for key, values in sample_characteristics.items():
    print(f"Row {key}: {values}")

# Based on available information, determine trait, age, and gender data
trait_row = None
age_row = None
gender_row = None

# Check each row in sample characteristics to identify relevant data
for key, values in sample_characteristics.items():
    # Convert values to strings for easier analysis
    str_values = [str(v).lower() if v is not None else "" for v in values]
    joined_values = " ".join(str_values).lower()
    
    # Look for anxiety-related terms
    if any(term in joined_values for term in ["anxiety", "anxious", "anx", "gad", "panic", "diagnosis", "condition", "disorder"]):
        trait_row = key
    
    # Look for age-related terms
    if any(term in joined_values for term in ["age", "years", "yr", "yrs"]):
        age_row = key
    
    # Look for gender-related terms
    if any(term in joined_values for term in ["gender", "sex", "male", "female"]):
        gender_row = key

# Define conversion functions
def convert_trait(value):
    if value is None:
        return None
    
    # Extract value after colon if present
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
    
    value_lower = str(value).lower()
    
    # Look for anxiety indicators
    if any(term in value_lower for term in ["anxiety", "anxious", "anxiety disorder", "gad", "panic"]):
        return 1
    elif any(term in value_lower for term in ["control", "healthy", "normal", "none"]):
        return 0
    else:
        return None

def convert_age(value):
    if value is None:
        return None
    
    # Extract value after colon if present
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Try to extract numeric age
    import re
    age_match = re.search(r'(\d+\.?\d*)', str(value))
    if age_match:
        return float(age_match.group(1))
    else:
        return None

def convert_gender(value):
    if value is None:
        return None
    
    # Extract value after colon if present
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
    
    value_lower = str(value).lower()
    
    if any(term in value_lower for term in ["female", "f", "woman", "women"]):
        return 0
    elif any(term in value_lower for term in ["male", "m", "man", "men"]):
        return 1
    else:
        return None

# 3. Save Metadata - Initial Filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait data is available and clinical data exists)
if trait_row is not None and clinical_data is not None:
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(